# TaxiSimulation examples

- typical import

In [1]:
using TaxiSimulationPotato, RoutingNetworksPotato, JLD

## Creating TaxiProblems

The `TaxiProblem` class represents problems we are trying to solve. Its most important features are:
- a `Network` object from the `RoutingNetworks` package, together with a `RoutingPaths` object from the same package. These types represent the static routing graph and directions that will be used by taxis. Another `RoutingPaths` object describes the costs. The hypothesis is that taxis will use shortest paths in time (and not in cost)
- a set of `Customers` (all customers to appear)
- a set of `Taxis` (all taxis on the map)

###  Synthetic problems

- We create here a typical small-sized routing problem on a synthetic city

In [3]:
import Random

In [4]:
Random.seed!(1)
# the network
n = urbanNetwork(8, distance=800.)

# the travel-times and paths. Here with road-types maximal speeds 
# (see RoutingNetworks for more details)

routing = roadTypeRouting(n)

#usually,the costs are proportional to the times (but not necessarily)
# here the cost is $5 per hour of driving
costs = RoutingPaths(n, routing.times*5. /3600.)

# We create the taxi problem
# - customerTime = nb of seconds to pickup or dropoff a customer
# - waitingCost = nb of $ per second of a taxi waiting. (here $1/hour)
pb = TaxiProblem(n,routing,costs,customerTime= 10., waitingCost = 1. /3600.)

# The problem is still incomplete, we need to add customer and taxis
# we first add random customers, see `taxiproblem/randomproblem.jl` for more info
# 1h of customer pickups, 0.35 customers per node per hour, fare = $80/hour, 
# customers can wait up to 5min after pickup time, and call 30min before
addRandomCustomers!(pb, 3600., 0.35, hourFare=80., custWait=5.0*60, custCall= 30.0*60)

# 20 uniformly distributed identical random taxis, all available at beginning of simulation
addRandomTaxis!(pb, 20)

# save this problem locally for latter use
save("data/smallurb.jld", "pb", pb)
pb

Taxi Problem
City with 192 locations and 640 roads
Simulation with 60 customers and 20 taxis for 60.00 minutes


In [5]:
print(pb)

Taxi Problem
City with 192 locations and 640 roads
Simulation with 60 customers and 20 taxis for 60.00 minutes


- We also create a harder medium-sized problem 

Random.seed!(1)
n = urbanNetwork(16, distance=300.)
r = roadTypeRouting(n)
costs = RoutingPaths(n,r.times*5. /3600.)
pb = TaxiProblem(n,r,costs,customerTime= 10., waitingCost = 1. /3600.)
addRandomCustomers!(pb, 3600., 0.35, hourFare=80., custWait=5.0*60, custCall= 30.0*60)
addRandomTaxis!(pb, 55)
pb = updateTcall(pb,60*20.0)
pb = updateTmax(pb, 60*5.0)
save("data/mediumurb.jld", "pb", pb)

### Real-world problems 

We present here the creation of a problem using Manhattan's taxi data

#### Constructing real-world networks
The first step is to create the desired routing network. This can be done with the package RoutingNetworks, from a polygon. In our case we will use the predefined MANHATTAN polygon.

In [6]:
MANHATTAN_POLY = Tuple{Float32,Float32}[(-74.01369f0,40.69977f0), (-74.00597f0,40.702637f0), (-73.99944f0,40.70641f0), (-73.991714f0,40.708492f0), (-73.9761f0,40.71044f0), (-73.96923f0,40.72931f0), (-73.973526f0,40.736073f0), (-73.9615f0,40.75402f0), (-73.941765f0,40.774693f0), (-73.94348f0,40.78223f0), (-73.938156f0,40.78535f0), (-73.93593f0,40.79029f0), (-73.928894f0,40.79432f0), (-73.92872f0,40.803024f0), (-73.93318f0,40.80744f0), (-73.9349f0,40.833942f0), (-73.92134f0,40.85745f0), (-73.91893f0,40.858356f0), (-73.913956f0,40.863678f0), (-73.909706f0,40.872345f0), (-73.91829f0,40.875168f0), (-73.92648f0,40.879192f0), (-73.93344f0,40.87244f0), (-73.933525f0,40.86793f0), (-73.943436f0,40.853584f0), (-73.947945f0,40.85164f0), (-73.94713f0,40.84414f0), (-73.9552f0,40.828682f0), (-73.96091f0,40.8205f0), (-73.97734f0,40.79864f0), (-73.98957f0,40.78077f0), (-73.996994f0,40.770725f0), (-74.00352f0,40.761368f0), (-74.01064f0,40.75103f0), (-74.01532f0,40.719486f0), (-74.01764f0,40.719063f0), (-74.02047f0,40.704067f0)];

- get data (can take some time)

In [7]:
manhattanNetwork = queryOsmPolygon(MANHATTAN_POLY)

┌ Warning: `Pkg.dir(pkgname, paths...)` is deprecated; instead, do `import RoutingNetworks; joinpath(dirname(pathof(RoutingNetworks)), "..", paths...)`.
└ @ Pkg C:\workdir\usr\share\julia\stdlib\v1.8\Pkg\src\Pkg.jl:687


SystemError: SystemError: opening file "nothing/.cache/data.osm": No such file or directory

- format the network to our need

In [8]:
manhattanNetwork = roadTypeSubset(manhattanNetwork,1:6) # only keep main driving roads
manhattanNetwork = stronglyConnected(manhattanNetwork,1) # only keep the main connected component (hoping it's the one of node 1)
manhattanNetwork = intersections(manhattanNetwork) # simplify enormously the network, so that each node correspond to an intersection

UndefVarError: UndefVarError: manhattanNetwork not defined

- save the network to a julia JLD file for latter use

In [9]:
saveTemplate(manhattanNetwork, "Manhattan")

UndefVarError: UndefVarError: manhattanNetwork not defined

- to load the network, just do:

In [10]:
manhattanNetwork = loadTemplate("Manhattan")
# visualize(manhattanNetwork)

ErrorException: template does not exist

In [11]:
visualize(manhattanNetwork)

UndefVarError: UndefVarError: manhattanNetwork not defined

#### Getting real taxi-data
For now, the package is able to parse and load CSVs of taxi trips from the NYC taxi and limousines commission. The CSVs can be found on their [website](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml). The data is parsed and files can then be saved by date to simplify the reading.

In [12]:
# Download data from website (will take some time, size = 1.8gb)
const DATA_URL_04_2016 = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-04.csv"
download(DATA_URL_04_2016, "data/taxidata042016.csv")

Downloads.RequestError: HTTP/1.1 403 Forbidden while requesting https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-04.csv

In [13]:
# Parse data
realcustomers = loadManhattanCustomers(manhattanNetwork, "data/taxidata042016.csv")

# Save to files (one per day)
saveByDate(realcustomers, "data/manhattan")

# clean a little bit
rm("data/taxidata042016.csv")

UndefVarError: UndefVarError: manhattanNetwork not defined

- Finally, let's create the taxi problem

In [14]:
# let's get some nice routing data created using another project
times = load("data/timingsestimates042016-0911-weekdays.jld", "times");
# if the previous file doe not exist, create virtual times instead:
# times = roadTypeRouting(manhattanNetwork).times

routing = RoutingPaths(manhattanNetwork, times)
costs = RoutingPaths(manhattanNetwork, routing.times*5./3600.)
pb = TaxiProblem(manhattanNetwork, routing, costs, customerTime=30., waitingCost = 1./3600.)

# Data from Wednesday 6th of April 2016 between 9 and 10:30
custData = load("data/manhattan-2016-04-06.jld", "customers")
startTime = DateTime(2016, 4, 6, 9, 0, 0)
endTime = startTime + Dates.Minute(90)

# adding 100% of the real demand
addDataCustomers!(pb, custData, startTime, endTime, 1.0)

# adding 3000 taxis with a distribution matching the customers pickups (to avoid "border effect")
addDistributedTaxis!(pb, 3000)

# updating customer behavior: they book between 0 and 20min earlier and wait up to 5min for a taxi.
pb = updateTcall(pb, 60*20.0,random=true)
pb = updateTmax(pb, 60*5.0)
save("data/manhattan.jld", "pb", pb)

pb

ArgumentError: ArgumentError: No file exists at given path: data/timingsestimates042016-0911-weekdays.jld

# Offline Solving
Offline solving is about finding the optimal solution when all customers are known beforehand.

## Optimization-based solving

In [15]:
using TaxiSimulationPotato, JLD

- We first solve the small problem to optimality, and visualize the result

In [16]:
#pb = load("data/smallurb.jld", "pb")
offlineSol = mipSolve(pb, verbose=false)

Set parameter Username
Solution metrics:
Revenues: $0.00, Costs: $20.00, Profit: $-20.00 
0.00% of the demand is met
In total, 0.00h of driving
100.00% of waiting time, NaN% of driving time is empty, 
Fairness: taxi profit = $-1.00 +- $0.00


Offline Solution, problem with 60 customers and 20 taxis
Profit : -20.00 dollars
60 customers rejected. 



mipSolve returns an `OfflineSolution` object. This object contains all the information to represent an offline solution. It basically contains the customers assigned to each taxi. A full `TaxiSolution` also contains all the taxi movements at any time. Full solutions can be used to visualize the result (though the conversion between the two types is implicit)
The commands of visualization are:
- `ARROWS` to move around
- `Z` and `X` to move in/out
- `A` and `S` to increase/decrease the drawing size
- `D` to show/hide the nodes of the network
- `Q` and `ESC` to quit
- `W` and `E` to slow down / accelerate time
- `R` to reverse time
- `SPACE` to pause time
- `click` to follow/unfollow a taxi

In [17]:
solution = TaxiSolution(offlineSol)
visualize(solution)

MethodError: MethodError: Cannot `convert` an object of type Ptr{CSFML.LibCSFML.sfCircleShape} to an object of type CSFML.LibCSFML.sfCircleShape
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T at Base.jl:61

The problem is internally converted to a network-flow problem in order to be solved. This formulation creates a graph where each node is a customer, and each directed edge represents a pair of customers that can be picked-up in a row.
When all feasible edges are given, the solution is the optimal one. On the other hand, one can solve the problem with fewer edges for more tractability at the cost of a less optimal solution. 

Note that a `FlowProblem` must be fully computed (all feasible pair of customers). The graph indicates what edges we do consider.


In [18]:
pb = load("data/mediumurb.jld", "pb")
flowPb = FlowProblem(pb)

# compute heuristic score for each edge. It's a rough indicator of how good an edge is.
scores = scoreHeuristic(pb, flowPb)
# only select the 3 best edges for each nodes, and 10 for the taxi origin nodes
smallerPb = kLinks(flowPb, 3, scores, firstK=10)

# scores are useful to avoid recomputing everything for each new K.
# but there is a one-liner :
smallerPb = kLinks(pb, 2, firstK=10)


flowSol = mipFlow(smallerPb, verbose=false)
sol = OfflineSolution(pb, smallerPb, flowSol)
# visualize(sol)

┌ Warning: type LightGraphs.SimpleGraphs.SimpleDiGraph{Core.Int64} not present in workspace; reconstructing
└ @ JLD C:\Users\Admin\.julia\packages\JLD\Vxkax\src\jld_types.jl:697
Error encountered while load FileIO.File{FileIO.DataFormat{:JLD}, String}("data/mediumurb.jld").

Fatal error:


CapturedException: MethodError: Cannot `convert` an object of type 
  JLD.var"##LightGraphs.SimpleGraphs.SimpleDiGraph{Core.Int64}#312" to an object of type 
  LightGraphs.SimpleGraphs.SimpleDiGraph
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T at Base.jl:61
Stacktrace:
  [1] macro expansion
    @ C:\Users\Admin\.julia\packages\JLD\Vxkax\src\jld_types.jl:372 [inlined]
  [2] jlconvert(#unused#::Type{Network}, file::JLD.JldFile, ptr::Ptr{UInt8})
    @ JLD C:\Users\Admin\.julia\packages\JLD\Vxkax\src\jld_types.jl:550
  [3] read_scalar_default(obj::JLD.JldDataset, dtype::HDF5.Datatype, T::Type)
    @ JLD C:\Users\Admin\.julia\packages\JLD\Vxkax\src\JLD.jl:419
  [4] read_scalar(obj::JLD.JldDataset, dtype::HDF5.Datatype, T::Type)
    @ JLD C:\Users\Admin\.julia\packages\JLD\Vxkax\src\JLD.jl:415
  [5] read(obj::JLD.JldDataset)
    @ JLD C:\Users\Admin\.julia\packages\JLD\Vxkax\src\JLD.jl:386
  [6] read_ref(f::JLD.JldFile, ref::HDF5.Reference)
    @ JLD C:\Users\Admin\.juli

In [19]:
sol

UndefVarError: UndefVarError: sol not defined

When pick-up times are fixed, the problem becomes an LP and we can use a (fast) LP solver.

In [20]:
# random pickup times (can also provide other time window)
times = randPickupTimes(flowPb)

# all pick-up as late as possible
times = fixedPickupTimes(flowPb, 1.)

sol = OfflineSolution(pb, flowPb, lpFlow(flowPb, times, verbose=false))

UndefVarError: UndefVarError: flowPb not defined

### Modifying a flow problem
It is possible to add links to a `FlowProblem` object:

In [21]:
# The easier way is to start with a fully precomputed `FlowProblem`:
flowPb = FlowProblem(pb)

# Then constructing a copy (share memory!) with the edges removed
newFlowPb = emptyFlow(flowPb)

# And add the edges of a computed solution:
flowSol = lpFlow(flowPb, fixedPickupTimes(flowPb), verbose=false)
addLinks!(newFlowPb, flowSol)

Set parameter Username


MethodError: MethodError: no method matching set_attributes(::JuMP.Model, ::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}})
Closest candidates are:
  set_attributes(::Union{MathOptInterface.OptimizerWithAttributes, JuMP.ConstraintRef, JuMP.GenericModel, JuMP.GenericVariableRef}, !Matched::Pair...) at C:\Users\Admin\.julia\packages\JuMP\V9nZm\src\optimizer_interface.jl:1428

## Heuristics
- Greedy insertions

In [22]:
pb = load("data/mediumurb.jld", "pb")
orderedInsertions(pb) # order = tmin

┌ Warning: type LightGraphs.SimpleGraphs.SimpleDiGraph{Core.Int64} not present in workspace; reconstructing
└ @ JLD C:\Users\Admin\.julia\packages\JLD\Vxkax\src\jld_types.jl:697
Error encountered while load FileIO.File{FileIO.DataFormat{:JLD}, String}("data/mediumurb.jld").

Fatal error:


CapturedException: MethodError: Cannot `convert` an object of type 
  JLD.var"##LightGraphs.SimpleGraphs.SimpleDiGraph{Core.Int64}#316" to an object of type 
  LightGraphs.SimpleGraphs.SimpleDiGraph
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T at Base.jl:61
Stacktrace:
  [1] macro expansion
    @ C:\Users\Admin\.julia\packages\JLD\Vxkax\src\jld_types.jl:372 [inlined]
  [2] jlconvert(#unused#::Type{Network}, file::JLD.JldFile, ptr::Ptr{UInt8})
    @ JLD C:\Users\Admin\.julia\packages\JLD\Vxkax\src\jld_types.jl:550
  [3] read_scalar_default(obj::JLD.JldDataset, dtype::HDF5.Datatype, T::Type)
    @ JLD C:\Users\Admin\.julia\packages\JLD\Vxkax\src\JLD.jl:419
  [4] read_scalar(obj::JLD.JldDataset, dtype::HDF5.Datatype, T::Type)
    @ JLD C:\Users\Admin\.julia\packages\JLD\Vxkax\src\JLD.jl:415
  [5] read(obj::JLD.JldDataset)
    @ JLD C:\Users\Admin\.julia\packages\JLD\Vxkax\src\JLD.jl:386
  [6] read_ref(f::JLD.JldFile, ref::HDF5.Reference)
    @ JLD C:\Users\Admin\.juli

- local improvement of greedy insertions order

In [23]:
insertionsDescent(pb)

ErrorException: setfield!: immutable struct of type CustomerTimeWindow cannot be changed

- best of random insertions orders

In [24]:
sol = randomInsertions(pb)

UndefVarError: UndefVarError: shuffle not defined

- local 2-OPT improvement of solution, with reinsertions at each step

In [25]:
localDescent(pb, sol, maxSearch=100, maxTime=20.)

UndefVarError: UndefVarError: sol not defined

- For very large problems (eg. Manhattan), smart-search tunes the maxSearch parameter in an online way.

In [26]:
smartSearch(pb,sol)

UndefVarError: UndefVarError: sol not defined

- Back-bone local search, uses LP to construct backbone, solve with MIP. Quite powerful

In [27]:
backboneSearch(pb, OfflineSolution(pb), maxEdges = 1500, maxTime=300, localityRatio = 0.75)

Initial profit: $0.00
Iteration 1: Set parameter Username


MethodError: MethodError: no method matching set_attributes(::JuMP.Model, ::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}})
Closest candidates are:
  set_attributes(::Union{MathOptInterface.OptimizerWithAttributes, JuMP.ConstraintRef, JuMP.GenericModel, JuMP.GenericVariableRef}, !Matched::Pair...) at C:\Users\Admin\.julia\packages\JuMP\V9nZm\src\optimizer_interface.jl:1428

## Online simulations

In [28]:
pb = load("data/mediumurb.jld", "pb")

┌ Warning: type LightGraphs.SimpleGraphs.SimpleDiGraph{Core.Int64} not present in workspace; reconstructing
└ @ JLD C:\Users\Admin\.julia\packages\JLD\Vxkax\src\jld_types.jl:697
Error encountered while load FileIO.File{FileIO.DataFormat{:JLD}, String}("data/mediumurb.jld").

Fatal error:


CapturedException: MethodError: Cannot `convert` an object of type 
  JLD.var"##LightGraphs.SimpleGraphs.SimpleDiGraph{Core.Int64}#320" to an object of type 
  LightGraphs.SimpleGraphs.SimpleDiGraph
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T at Base.jl:61
Stacktrace:
  [1] macro expansion
    @ C:\Users\Admin\.julia\packages\JLD\Vxkax\src\jld_types.jl:372 [inlined]
  [2] jlconvert(#unused#::Type{Network}, file::JLD.JldFile, ptr::Ptr{UInt8})
    @ JLD C:\Users\Admin\.julia\packages\JLD\Vxkax\src\jld_types.jl:550
  [3] read_scalar_default(obj::JLD.JldDataset, dtype::HDF5.Datatype, T::Type)
    @ JLD C:\Users\Admin\.julia\packages\JLD\Vxkax\src\JLD.jl:419
  [4] read_scalar(obj::JLD.JldDataset, dtype::HDF5.Datatype, T::Type)
    @ JLD C:\Users\Admin\.julia\packages\JLD\Vxkax\src\JLD.jl:415
  [5] read(obj::JLD.JldDataset)
    @ JLD C:\Users\Admin\.julia\packages\JLD\Vxkax\src\JLD.jl:386
  [6] read_ref(f::JLD.JldFile, ref::HDF5.Reference)
    @ JLD C:\Users\Admin\.juli

Online simulation relies on the `onlineSimulation` function, that takes a taxi problem and and feeds it to an online algorithm. Either updates the solution at a predefined time-steps or for each new customer.

A general class of online algorithm are offline-planning, that maintain an inner offline solution of future moves throughout the simulation

- The simplest one is `InsertOnly`, that just insert new customers into a taxi's timeline.

In [29]:
algo = InsertOnly(earliest=false)
sol = onlineSimulation(pb, algo)

# a horizon can be added:
sol = onlineSimulation(pb, algo, horizon=60*20, verbose=false, period=30)

pre-simulation computations...

MethodError: MethodError: no method matching (Array{Customer})(::Int64)
Closest candidates are:
  (Array{T})(!Matched::LinearAlgebra.AbstractQ) where T at C:\Users\Admin\.julia\juliaup\julia-1.8.5+0.x64.w64.mingw32\share\julia\stdlib\v1.8\LinearAlgebra\src\qr.jl:577
  (Array{T})(!Matched::LinearAlgebra.LQPackedQ{T, S, C} where {S<:AbstractMatrix{T}, C<:AbstractVector{T}}) where T at C:\Users\Admin\.julia\juliaup\julia-1.8.5+0.x64.w64.mingw32\share\julia\stdlib\v1.8\LinearAlgebra\src\lq.jl:171
  (Array{T})(!Matched::StaticArraysCore.SizedArray{S, T, N, M, TData} where {N, M, TData<:AbstractArray{T, M}}) where {T, S} at C:\Users\Admin\.julia\packages\StaticArrays\9Yt0H\src\SizedArray.jl:63
  ...

- `NearestTaxi` assign the nearest free taxi to new customers, at the time they appear (pure online)

In [30]:
algo = NearestTaxi(freeTaxiOnly=false)
sol = onlineSimulation(pb, algo)

pre-simulation computations...

MethodError: MethodError: no method matching (Array{Customer})(::Int64)
Closest candidates are:
  (Array{T})(!Matched::LinearAlgebra.AbstractQ) where T at C:\Users\Admin\.julia\juliaup\julia-1.8.5+0.x64.w64.mingw32\share\julia\stdlib\v1.8\LinearAlgebra\src\qr.jl:577
  (Array{T})(!Matched::LinearAlgebra.LQPackedQ{T, S, C} where {S<:AbstractMatrix{T}, C<:AbstractVector{T}}) where T at C:\Users\Admin\.julia\juliaup\julia-1.8.5+0.x64.w64.mingw32\share\julia\stdlib\v1.8\LinearAlgebra\src\lq.jl:171
  (Array{T})(!Matched::StaticArraysCore.SizedArray{S, T, N, M, TData} where {N, M, TData<:AbstractArray{T, M}}) where {T, S} at C:\Users\Admin\.julia\packages\StaticArrays\9Yt0H\src\SizedArray.jl:63
  ...

- `SearchBudget` is a heuristic-based algorithm that uses an offline solver to continuously improve the solution. The time_budget parameter controls the percentage of "simulation real-time" that is used to find a better solution. This method can use any offline solver.

In [31]:
algo = SearchBudget(time_budget = 0.1)
sol = onlineSimulation(pb, algo)

pre-simulation computations...

MethodError: MethodError: no method matching (Array{Customer})(::Int64)
Closest candidates are:
  (Array{T})(!Matched::LinearAlgebra.AbstractQ) where T at C:\Users\Admin\.julia\juliaup\julia-1.8.5+0.x64.w64.mingw32\share\julia\stdlib\v1.8\LinearAlgebra\src\qr.jl:577
  (Array{T})(!Matched::LinearAlgebra.LQPackedQ{T, S, C} where {S<:AbstractMatrix{T}, C<:AbstractVector{T}}) where T at C:\Users\Admin\.julia\juliaup\julia-1.8.5+0.x64.w64.mingw32\share\julia\stdlib\v1.8\LinearAlgebra\src\lq.jl:171
  (Array{T})(!Matched::StaticArraysCore.SizedArray{S, T, N, M, TData} where {N, M, TData<:AbstractArray{T, M}}) where {T, S} at C:\Users\Admin\.julia\packages\StaticArrays\9Yt0H\src\SizedArray.jl:63
  ...

In [32]:
algo = SearchBudget(
update_solver     = (pb,init,custs,t)-> mipSolve(pb, FlowProblem(pb, collect(custs)), Nullable{OfflineSolution}(init), verbose=true),
precompute_solver = (pb,custs)     -> mipSolve(pb, FlowProblem(pb, collect(custs)), Nullable{OfflineSolution}(), verbose=true),
    time_budget=1.,  update_freq=60.
)

sol = onlineSimulation(pb, algo, horizon=60*20, period=30.)

pre-simulation computations...

MethodError: MethodError: no method matching (Array{Customer})(::Int64)
Closest candidates are:
  (Array{T})(!Matched::LinearAlgebra.AbstractQ) where T at C:\Users\Admin\.julia\juliaup\julia-1.8.5+0.x64.w64.mingw32\share\julia\stdlib\v1.8\LinearAlgebra\src\qr.jl:577
  (Array{T})(!Matched::LinearAlgebra.LQPackedQ{T, S, C} where {S<:AbstractMatrix{T}, C<:AbstractVector{T}}) where T at C:\Users\Admin\.julia\juliaup\julia-1.8.5+0.x64.w64.mingw32\share\julia\stdlib\v1.8\LinearAlgebra\src\lq.jl:171
  (Array{T})(!Matched::StaticArraysCore.SizedArray{S, T, N, M, TData} where {N, M, TData<:AbstractArray{T, M}}) where {T, S} at C:\Users\Admin\.julia\packages\StaticArrays\9Yt0H\src\SizedArray.jl:63
  ...

`BackbonePlanning` is a full optimization online method based on the backbone local optimization. It updates real time a graph representing the problem and constantly search for a better solution. Lots of parameters make it possible to control the space restriction, the computational time allowed, etc...

In [33]:
algo = BackbonePlanning(edgesPerNode=20, precompTime=60, iterTime=5, maxEdges = 400, maxExplorationTime = 1)
sol = onlineSimulation(pb, algo, horizon=60*20, verbose=true, period=30)

pre-simulation computations...

MethodError: MethodError: no method matching (Array{Tuple{Float64, Float64}})(::Int64)
Closest candidates are:
  (Array{T})(!Matched::LinearAlgebra.AbstractQ) where T at C:\Users\Admin\.julia\juliaup\julia-1.8.5+0.x64.w64.mingw32\share\julia\stdlib\v1.8\LinearAlgebra\src\qr.jl:577
  (Array{T})(!Matched::LinearAlgebra.LQPackedQ{T, S, C} where {S<:AbstractMatrix{T}, C<:AbstractVector{T}}) where T at C:\Users\Admin\.julia\juliaup\julia-1.8.5+0.x64.w64.mingw32\share\julia\stdlib\v1.8\LinearAlgebra\src\lq.jl:171
  (Array{T})(!Matched::StaticArraysCore.SizedArray{S, T, N, M, TData} where {N, M, TData<:AbstractArray{T, M}}) where {T, S} at C:\Users\Admin\.julia\packages\StaticArrays\9Yt0H\src\SizedArray.jl:63
  ...